# Venue Visit Analysis

This notebook performs an analysis to identify which venues from `filtered_data.csv` were visited by individuals based on mobility pings from `MobilityDataMay2024.parquet`.

**Steps:**
1. Load mobility data and venue data.
2. Convert both to GeoDataFrames.
3. Create a 50-meter buffer around each venue.
4. Perform a spatial join to find mobility pings falling within these venue buffers.
5. Save the results.

## 1. Setup and Library Imports

In [ ]:
import pandas as pd
import geopandas
from shapely.geometry import Point
import pyarrow # Required for parquet
import os

print("Libraries imported.")

### Mount Google Drive (if running in Colab)

In [ ]:
try:
    from google.colab import drive
    drive.mount('/content/drive')
    print("Google Drive mounted.")
    # Define base path for files on Google Drive
    # IMPORTANT: Adjust this path if your files are in a subfolder of MyDrive
    google_drive_base_path = '/content/drive/MyDrive/'
except ModuleNotFoundError:
    print("Not running in Colab, or google.colab module not found. Assuming files are local.")
    # Define a base path for local files (current directory)
    google_drive_base_path = './' # Current directory if not in Colab
except Exception as e:
    print(f"Error mounting Google Drive: {e}")
    google_drive_base_path = './'

## 2. Helper Function for Coordinate Cleaning

In [ ]:
def clean_coordinates(coord_series):
    """Cleans coordinate strings by replacing commas with periods and converting to numeric."""
    return pd.to_numeric(coord_series.astype(str).str.replace(',', '.', regex=False), errors='coerce')

print("Helper function defined.")

## 3. Load Mobility Data

In [ ]:
print("Loading mobility data (MobilityDataMay2024.parquet)...")
# Construct path using the google_drive_base_path variable
mobility_data_filename = 'MobilityDataMay2024.parquet'
mobility_data_path = os.path.join(google_drive_base_path, mobility_data_filename)
df_mobility = None
try:
    # Load the entire parquet file first
    df_mobility_full = pd.read_parquet(mobility_data_path)
    print(f"Full mobility data loaded. Shape: {df_mobility_full.shape}")
    
    # Take the first 1 million rows
    num_rows_to_sample = 1000000
    if len(df_mobility_full) > num_rows_to_sample:
        df_mobility = df_mobility_full.head(num_rows_to_sample)
        print(f"Using the first {num_rows_to_sample} rows of mobility data. New shape: {df_mobility.shape}")
    else:
        df_mobility = df_mobility_full
        print(f"Full mobility data has {len(df_mobility_full)} rows (less than or equal to 1 million), using all of it. Shape: {df_mobility.shape}")
    del df_mobility_full # Free up memory

    # Basic validation
    if not all(col in df_mobility.columns for col in ['latitude', 'longitude', 'device_aid', 'timestamp']):
        print("Error: Mobility data is missing one or more required columns: 'latitude', 'longitude', 'device_aid', 'timestamp'.")
        df_mobility = None # Invalidate df_mobility
    else:
        print("Required columns found in mobility data.")
        print(df_mobility.head())
except FileNotFoundError:
    print(f"Error: Mobility data file not found at {mobility_data_path}")
except Exception as e:
    print(f"Error loading mobility data: {e}")

## 4. Create Mobility GeoDataFrame

In [ ]:
gdf_mobility = None
if df_mobility is not None:
    print("Creating mobility GeoDataFrame...")
    try:
        gdf_mobility = geopandas.GeoDataFrame(
            df_mobility,
            geometry=geopandas.points_from_xy(df_mobility.longitude, df_mobility.latitude),
            crs="EPSG:4326"  # WGS84
        )
        print(f"Mobility GeoDataFrame created. Shape: {gdf_mobility.shape}, CRS: {gdf_mobility.crs}")
        print(gdf_mobility.head())
    except Exception as e:
        print(f"Error creating mobility GeoDataFrame: {e}")
else:
    print("Skipping mobility GeoDataFrame creation as df_mobility was not loaded.")

## 5. Load Venue Data

In [ ]:
print("Loading venue data (filtered_data.csv)...")
# Construct path using the google_drive_base_path variable
venue_data_filename = 'filtered_data.csv'
venue_data_path = os.path.join(google_drive_base_path, venue_data_filename)
df_venues = None
try:
    df_venues = pd.read_csv(venue_data_path, sep=';')
    print(f"Venue data loaded. Shape: {df_venues.shape}")
    # Basic validation
    if not all(col in df_venues.columns for col in ['lat', 'lng', 'MusteriKodu']):
        print("Error: Venue data is missing one or more required columns: 'lat', 'lng', 'MusteriKodu'.")
        df_venues = None # Invalidate df_venues
    else:
        print("Required columns found in venue data.")
        print(df_venues.head())
except FileNotFoundError:
    print(f"Error: Venue data file not found at {venue_data_path}")
except Exception as e:
    print(f"Error loading venue data: {e}")

## 6. Clean Venue Coordinates and Create Venue GeoDataFrame

In [ ]:
gdf_venues = None
if df_venues is not None:
    print("Cleaning venue coordinates and creating venue GeoDataFrame...")
    try:
        df_venues['lat_cleaned'] = clean_coordinates(df_venues['lat'])
        df_venues['lng_cleaned'] = clean_coordinates(df_venues['lng'])
        
        # Drop rows with invalid coordinates
        original_venue_count = len(df_venues)
        df_venues.dropna(subset=['lat_cleaned', 'lng_cleaned'], inplace=True)
        print(f"Dropped {original_venue_count - len(df_venues)} venues due to invalid coordinates.")

        if df_venues.empty:
            print("Error: No valid venue coordinates after cleaning.")
        else:
            gdf_venues = geopandas.GeoDataFrame(
                df_venues,
                geometry=geopandas.points_from_xy(df_venues.lng_cleaned, df_venues.lat_cleaned),
                crs="EPSG:4326"  # WGS84
            )
            print(f"Venue GeoDataFrame created. Shape: {gdf_venues.shape}, CRS: {gdf_venues.crs}")
            print(gdf_venues.head())
    except Exception as e:
        print(f"Error creating venue GeoDataFrame: {e}")
else:
    print("Skipping venue GeoDataFrame creation as df_venues was not loaded.")

## 7. Spatial Analysis: Buffering and Joining

In [ ]:
gdf_visits = None
if gdf_venues is not None and gdf_mobility is not None:
    # Target CRS for buffering and spatial join (UTM Zone 36N, suitable for Istanbul/Western Turkey)
    # If data covers a wider area, a more dynamic UTM zone selection or a different projected CRS might be needed.
    projected_crs = "EPSG:32636" 
    buffer_radius_meters = 50

    print(f"Projecting venue data to {projected_crs} for buffering...")
    try:
        gdf_venues_projected = gdf_venues.to_crs(projected_crs)
        print(f"Venue data projected. CRS: {gdf_venues_projected.crs}")
    except Exception as e:
        print(f"Error projecting venue data: {e}")
        gdf_venues_projected = None

    gdf_venue_buffers = None
    if gdf_venues_projected is not None:
        print(f"Creating {buffer_radius_meters}m buffers around venues...")
        try:
            # Ensure the geometry column is active and valid before buffering
            if not gdf_venues_projected.geometry.is_valid.all():
                print("Warning: Some venue geometries are invalid. Attempting to fix...")
                # A common trick to fix invalid geometries; may not always work perfectly.
                gdf_venues_projected.geometry = gdf_venues_projected.geometry.buffer(0) 
                if not gdf_venues_projected.geometry.is_valid.all():
                     print("Error: Could not fix all invalid venue geometries. Proceeding with potentially problematic data.")
            
            gdf_venue_buffers = gdf_venues_projected.copy()
            gdf_venue_buffers['geometry'] = gdf_venues_projected.geometry.buffer(buffer_radius_meters)
            print(f"Venue buffers created. Shape: {gdf_venue_buffers.shape}")
        except Exception as e:
            print(f"Error creating venue buffers: {e}")
            gdf_venue_buffers = None

    gdf_mobility_projected = None
    if gdf_venue_buffers is not None: # Proceed only if buffers were created
        print(f"Projecting mobility data to {projected_crs} for spatial join...")
        try:
            gdf_mobility_projected = gdf_mobility.to_crs(projected_crs)
            print(f"Mobility data projected. CRS: {gdf_mobility_projected.crs}")
        except Exception as e:
            print(f"Error projecting mobility data: {e}")
            gdf_mobility_projected = None

    if gdf_mobility_projected is not None and gdf_venue_buffers is not None:
        print("Performing spatial join (mobile pings within venue buffers)...")
        try:
            # Use all columns from gdf_venue_buffers for the join to retain all venue information
            gdf_venue_buffers_for_join = gdf_venue_buffers.copy()
            print(f"Columns in gdf_venue_buffers_for_join before sjoin: {gdf_venue_buffers_for_join.columns.tolist()}")
            
            # Perform the spatial join
            # 'predicate="within"' means mobility points must be within venue buffers
            gdf_visits = geopandas.sjoin(gdf_mobility_projected, gdf_venue_buffers_for_join, how='inner', predicate='within')
            print(f"Spatial join completed. Number of potential visit pings: {gdf_visits.shape[0]}")
            if gdf_visits.empty:
                print("No visits found after spatial join.")
            else:
                print("Sample of joined visit data (first 5 rows):")
                print(gdf_visits.head())
        except Exception as e:
            print(f"Error during spatial join: {e}")
            gdf_visits = None
else:
    print("Skipping spatial analysis as one or both GeoDataFrames (mobility, venues) are missing.")

## 8. Process Results and Save Output

In [ ]:
if gdf_visits is not None and not gdf_visits.empty:
    print("Extracting relevant columns for the final output...")
    
    # gdf_visits index comes from gdf_mobility_projected, which in turn comes from gdf_mobility (and df_mobility)
    # We can use this index to retrieve original lat/lon from the initial df_mobility
    df_visits_output = gdf_visits.copy()
    
    # Check if the index of df_visits_output is present in df_mobility.index
    # This assumes that the index was preserved through GeoDataFrame conversions and projections.
    # If sampling or other index-altering operations were done on df_mobility before creating gdf_mobility,
    # this direct index lookup might fail or be incorrect.
    # For this script, we assume the index is consistent.
    try:
        df_visits_output['original_latitude'] = df_mobility.loc[df_visits_output.index, 'latitude'].values
        df_visits_output['original_longitude'] = df_mobility.loc[df_visits_output.index, 'longitude'].values
    except KeyError as e:
        print(f"KeyError while trying to map original lat/lon: {e}. Original coordinates might be missing.")
        df_visits_output['original_latitude'] = pd.NA
        df_visits_output['original_longitude'] = pd.NA
    except Exception as e:
        print(f"An unexpected error occurred while mapping original lat/lon: {e}")
        df_visits_output['original_latitude'] = pd.NA
        df_visits_output['original_longitude'] = pd.NA

    # Columns from mobility data (device_aid, timestamp, and the original lat/lon of the ping)
    mobility_cols_to_keep = ['device_aid', 'timestamp', 'original_latitude', 'original_longitude']

    # Original columns from df_venues (loaded from filtered_data.csv)
    # Exclude helper columns we added like 'lat_cleaned', 'lng_cleaned', and the 'geometry' column itself from gdf_venues
    # df_venues is the DataFrame loaded from filtered_data.csv and subsequently used to create gdf_venues
    original_venue_column_names = [col for col in df_venues.columns if col not in ['lat', 'lng', 'lat_cleaned', 'lng_cleaned', 'geometry']]
    
    processed_venue_columns_for_output = []
    # df_visits_output contains columns from gdf_mobility_projected and gdf_venue_buffers_for_join (which has all original venue columns)
    # We need to handle potential suffixes added by sjoin, typically '_right' for columns from the right GeoDataFrame (gdf_venue_buffers_for_join)
    for venue_col_original_name in original_venue_column_names:
        # Check if the original name exists (it might if it wasn't duplicated in gdf_mobility_projected)
        if venue_col_original_name in df_visits_output.columns:
            processed_venue_columns_for_output.append(venue_col_original_name)
        # Check if the suffixed version exists
        elif f"{venue_col_original_name}_right" in df_visits_output.columns:
            # Rename the suffixed column to its original name for cleaner output
            df_visits_output.rename(columns={f"{venue_col_original_name}_right": venue_col_original_name}, inplace=True)
            processed_venue_columns_for_output.append(venue_col_original_name)
        # If neither exists, it means the column was not in the sjoin result (e.g. if it was all NaNs and dropped, or not selected for join - though we select all now)

    # Combine mobility columns and all successfully processed (and renamed) venue columns
    # Ensure 'geometry' from the venue side (if present and suffixed) is not included unless explicitly desired
    # Also, 'index_right' is an sjoin artifact we usually don't need in the final output.
    # The 'geometry' column in df_visits_output is from the mobility pings (left side of sjoin).
    final_output_columns_list = mobility_cols_to_keep + processed_venue_columns_for_output
    
    # Ensure all selected columns actually exist in df_visits_output to prevent KeyErrors,
    # and remove duplicates if any column name was accidentally in both lists or processed multiple times.
    # Also, remove helper columns that might have slipped through from the venue side if they were named like original data columns.
    columns_to_exclude_finally = ['geometry_right', 'index_right', 'lat_cleaned', 'lng_cleaned'] # geometry_right might appear if venue geometry was also named 'geometry'
    
    final_selected_columns = []
    seen_columns = set()
    for col in final_output_columns_list:
        if col in df_visits_output.columns and col not in columns_to_exclude_finally and col not in seen_columns:
            final_selected_columns.append(col)
            seen_columns.add(col)
            
    print(f"Final columns selected for output: {final_selected_columns}")
        
    df_final_visits = df_visits_output[final_selected_columns]

    output_filename = 'device_venue_visits.csv'
    print(f"Saving results to {output_filename}...")
    try:
        df_final_visits.to_csv(output_filename, index=False, sep=';')
        print(f"Successfully saved results to {output_filename}. Shape: {df_final_visits.shape}")
        print(df_final_visits.head())
    except Exception as e:
        print(f"Error saving results: {e}")
else:
    print("No visit data to save (either gdf_visits is None or empty).")

print("Venue visit analysis script finished.")

## 9. Load Additional Device Features and Merge

Load `device_venue_visits.csv` and `device_featuresv2.csv`, then merge them on `device_aid`.

In [ ]:
print("Loading device_venue_visits.csv...")
visits_df = None
try:
    visits_df = pd.read_csv('device_venue_visits.csv', sep=';')
    print(f"device_venue_visits.csv loaded. Shape: {visits_df.shape}")
    print(visits_df.head())
    if 'device_aid' not in visits_df.columns:
        print("Error: 'device_aid' column not found in device_venue_visits.csv")
        visits_df = None
except FileNotFoundError:
    print("Error: device_venue_visits.csv not found.")
except Exception as e:
    print(f"Error loading device_venue_visits.csv: {e}")

In [ ]:
print("Loading device_featuresv2.csv...")
features_df = None
features_filename = 'device_featuresv2.csv'
features_path = os.path.join(google_drive_base_path, features_filename)
try:
    features_df = pd.read_csv(features_path) # Assuming standard comma separator
    print(f"{features_filename} loaded from {features_path}. Shape: {features_df.shape}")
    
    # If the first column is named 'Unnamed: 0' (pandas default for blank header), rename it to 'device_aid'
    if not features_df.empty and features_df.columns[0] == 'Unnamed: 0':
        features_df.rename(columns={'Unnamed: 0': 'device_aid'}, inplace=True)
        print("Renamed first column from 'Unnamed: 0' to 'device_aid'.")
    
    print(features_df.head())
    
    if 'device_aid' not in features_df.columns:
        print(f"Error: 'device_aid' column not found or not correctly named in {features_filename}. Please check CSV header.")
        features_df = None
except FileNotFoundError:
    print(f"Error: {features_filename} not found at {features_path}")
except Exception as e:
    print(f"Error loading {features_filename}: {e}")

In [ ]:
merged_df = None
if visits_df is not None and features_df is not None:
    print("Merging dataframes on 'device_aid'...")
    try:
        # Ensure device_aid is of the same type if necessary, though pandas often handles this.
        # For example, if one is int and other is string: 
        # visits_df['device_aid'] = visits_df['device_aid'].astype(str)
        # features_df['device_aid'] = features_df['device_aid'].astype(str)
        merged_df = pd.merge(visits_df, features_df, on='device_aid', how='inner')
        print(f"Merge successful. Shape of merged_df: {merged_df.shape}")
        print("Merged DataFrame head:")
        print(merged_df.head())
    except Exception as e:
        print(f"Error during merge: {e}")
else:
    print("Skipping merge as one or both dataframes were not loaded successfully.")

In [ ]:
if merged_df is not None and not merged_df.empty:
    output_merged_filename = 'mobil_restaurant.csv'
    print(f"Saving merged data to {output_merged_filename}...")
    try:
        merged_df.to_csv(output_merged_filename, index=False, sep=';')
        print(f"Successfully saved merged data to {output_merged_filename}. Shape: {merged_df.shape}")
    except Exception as e:
        print(f"Error saving merged data: {e}")
else:
    print("No merged data to save.")